In [ ]:
include("_.jl")

te = BioLab.Path.make_temporary("BioLab.Match.test")

using Distances

function get_correlation(nu1_, nu2_)

    return 1 - CorrDist()(nu1_, nu2_)

end

fu = get_correlation

function benchmark(n_fe, n_sa, ho)

    tan = "Target"

    sa1_ = ["Sample $id" for id in 1:n_sa]

    if ho == "1.0:"

        ta_ = collect(1.0:n_sa)

    elseif ho == "randn"

        ta_ = randn(n_sa)

    end

    fen = "Feature"

    fe_ = ["Feature $id" for id in 1:n_fe]

    fe_x_sa_x_nu = BioLab.Matrix.simulate(n_fe, n_sa, ho; re = 1)

    tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu

end

In [ ]:
n_fe, n_sa = (2, 4)

tan, sa1_, taf_, fen, fe_, fe_x_sa_x_fl = benchmark(n_fe, n_sa, "1.0:")

tai_ = convert(Vector{Int}, taf_)

fe_x_sa_x_in = convert(Matrix{Int}, fe_x_sa_x_fl)

for (ta_, fe_x_sa_x_nu, title_text) in (
    (taf_, fe_x_sa_x_fl, "Float x Float"),
    (tai_, fe_x_sa_x_fl, "Int x Float"),
    (taf_, fe_x_sa_x_in, "Float x Int"),
    (tai_, fe_x_sa_x_in, "Int x Int"),
)

    layout = Dict("title" => Dict("text" => title_text))

    BioLab.Match.make(fu, tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu; layout)

end

In [ ]:
n_fe, n_sa = (2, 4)

for ic in (true, false)

    tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu = benchmark(n_fe, n_sa, "1.0:")

    layout = Dict("title" => Dict("text" => "$(ifelse(ic, "Increasing", "Decreasing")) Target"))

    BioLab.Match.make(fu, tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu; ic, layout)

end

In [ ]:
n_fe, n_sa = (5, 2)

for n_ex in 0:(n_fe + 1)

    tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu = benchmark(n_fe, n_sa, "1.0:")

    layout = Dict("title" => Dict("text" => BioLab.String.count_noun(n_ex, "Extreme")))

    BioLab.Match.make(fu, tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu; n_ex, layout)

end

In [ ]:
n_fe, n_sa = (2, 4)

for st in (0, 0.1, 1, 2, 4)

    tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu = benchmark(n_fe, n_sa, "randn")

    layout = Dict("title" => Dict("text" => BioLab.String.count_noun(st, "Standard Deviation")))

    BioLab.Match.make(fu, tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu; st, layout)

end

In [ ]:
for (n_fe, n_sa) in ((2, 2), (4, 4), (8, 8), (16, 16), (80, 80), (1000, 4), (4, 1000))

    tan, sa1_, ta_, fen, fe_, fe_x_sa_x_nu = benchmark(n_fe, n_sa, "randn")

    ti = "$(n_fe) x $(n_sa)"

    layout = Dict("title" => Dict("text" => ti))

    BioLab.Match.make(
        fu,
        tan,
        sa1_,
        ta_,
        fen,
        fe_,
        fe_x_sa_x_nu;
        layout,
        ht = joinpath(te, "$(BioLab.Path.clean(ti)).html"),
    )

end